# AC109 Project: Predicting the returns on Cryptocurrencies

by Ali Dastjerdi, Angelina Massa, Sachin Mathur & Nate Stein

In [1]:
import crypto_utils as crypu
import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys

from crypto_utils import fmt_date, print_update
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

In [10]:
# Custom output options.

np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.precision', 4)
sns.set_style('white')
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.pad_inches'] = 0.05
%matplotlib inline

In [3]:
x_cryptos = ['ltc', 'xrp', 'xlm', 'eth']
n_rolling_price = 1
n_rolling_volume = 2
y_crypto = 'btc'
x_assets = ['SP500']
dm = crypu.DesignMatrix(
    x_cryptos=x_cryptos, y_crypto=y_crypto, x_assets=x_assets,
    n_rolling_price=n_rolling_price, n_rolling_volume=n_rolling_volume)

In [4]:
dm.load_time_series()
df = dm.df.copy()

In [5]:
df.head()

,ltc,ltc_volume,xrp,xrp_volume,xlm,xlm_volume,eth,eth_volume,btc,btc_volume,SP500
2015-08-08,-0.086,0.268,0.040,0.806,-0.005,2.136,-0.728,0.000,-0.066,2.115,-2.875e-03
2015-08-09,0.013,-0.269,0.039,0.463,-0.031,2.159,-0.068,2.238,0.016,-0.440,-2.875e-03
2015-08-10,0.013,-0.545,-0.007,-0.303,0.052,-0.224,0.009,-0.399,-0.002,-0.642,1.281e-02
2015-08-11,0.053,0.118,-0.018,-0.469,-0.028,-0.441,0.510,1.749,0.022,0.069,-9.557e-03
2015-08-12,-0.041,0.571,-0.037,-0.392,-0.008,0.233,0.140,4.306,-0.015,0.278,9.501e-04


In [32]:
GLOBAL_TEST = 0

def roll_func(x):
    recent_q = x[-1]
    nonrecent_q = x[0:-1]
    global GLOBAL_TEST
    if GLOBAL_TEST <= 2:
        print('Recent: {}'.format(recent_q))
        print('Nonrecent: {}'.format(nonrecent_q))
    GLOBAL_TEST += 1
    return recent_q / sum(nonrecent_q)
    
#df['ltc_ma'] = df['ltc'].shift(1).rolling(window=20).mean()
#df_calc = df.shift(1).rolling(window=20).mean()
df['qtest_ma'] = df['qtest'].rolling(window=21).apply(roll_func)

Recent: 20.0
Nonrecent: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.]
Recent: 21.0
Nonrecent: [  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.]
Recent: 22.0
Nonrecent: [  2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.
  17.  18.  19.  20.  21.]


In [33]:
test_idx = pd.to_datetime('9/1/2015')
df.loc[test_idx]

ltc           -0.0070
ltc_volume    -0.0811
xrp           -0.0088
xrp_volume    -0.0169
xlm           -0.0177
xlm_volume    -0.8945
eth           -0.0074
eth_volume    -0.4177
btc           -0.0084
btc_volume     0.0599
SP500         -0.0296
ltc_ma         1.0000
qtest         24.0000
qtest_ma       0.0889
Name: 2015-09-01 00:00:00, dtype: float64

In [26]:
df['qtest'] = range(df.shape[0])